In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,39791
2,Huelva,Confirmados PDIA 14 días,3101
3,Huelva,Tasa PDIA 14 días,"604,2831810121402"
4,Huelva,Confirmados PDIA 7 días,1402
5,Huelva,Tasa PDIA 7 dias,"273,203811602393"
6,Huelva,Total Confirmados,39999
7,Huelva,Curados,34618
8,Huelva,Fallecidos,395


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  39791.0


/tmp/ipykernel_3327/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  11100.0


/tmp/ipykernel_3327/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3327/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3327/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3327/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 376 personas en los últimos 7 días 

Un positivo PCR cada 166 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,39791.0,1402.0,3101.0,513170.0,273.203812,604.283181,272.0
Huelva-Costa,23376.0,822.0,1863.0,289548.0,283.890754,643.416636,135.0
Condado-Campiña,12469.0,435.0,871.0,156231.0,278.433858,557.507793,99.0
Sierra de Huelva-Andévalo Central,3566.0,140.0,360.0,67391.0,207.742874,534.195961,37.0
Lepe,2611.0,87.0,201.0,27880.0,312.051650,720.946915,31.0
Huelva (capital),11100.0,383.0,867.0,143837.0,266.273629,602.765631,27.0
Isla Cristina,2703.0,90.0,194.0,21393.0,420.698359,906.838686,20.0
Punta Umbría,1129.0,78.0,167.0,15355.0,507.977857,1087.593618,18.0
Gibraleón,835.0,53.0,112.0,12737.0,416.110544,879.327942,15.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cumbres Mayores,113.0,31.0,54.0,1749.0,1772.441395,3087.478559,5.0
Encinasola,159.0,7.0,33.0,1310.0,534.351145,2519.083969,0.0
Corteconcepción,35.0,1.0,11.0,536.0,186.567164,2052.238806,1.0
Cumbres de Enmedio,1.0,1.0,1.0,51.0,1960.784314,1960.784314,0.0
Cañaveral de León,11.0,3.0,6.0,396.0,757.575758,1515.151515,0.0
Sanlúcar de Guadiana,12.0,5.0,5.0,403.0,1240.694789,1240.694789,2.0
Beas,318.0,20.0,53.0,4341.0,460.723336,1220.916839,3.0
Nava (La),5.0,2.0,3.0,258.0,775.193798,1162.790698,2.0
Punta Umbría,1129.0,78.0,167.0,15355.0,507.977857,1087.593618,18.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Corteconcepción,35.0,1.0,11.0,536.0,186.567164,2052.238806,1.0,0.090909
Higuera de la Sierra,44.0,1.0,7.0,1291.0,77.459334,542.215337,0.0,0.142857
Rosal de la Frontera,44.0,1.0,6.0,1697.0,58.927519,353.565115,1.0,0.166667
Cortegana,261.0,5.0,26.0,4602.0,108.648414,564.971751,0.0,0.192308
Galaroza,49.0,1.0,5.0,1382.0,72.358900,361.794501,1.0,0.200000
Encinasola,159.0,7.0,33.0,1310.0,534.351145,2519.083969,0.0,0.212121
Bollullos Par del Condado,1120.0,15.0,46.0,14387.0,104.260791,319.733092,6.0,0.326087
Campillo (El),84.0,1.0,3.0,2024.0,49.407115,148.221344,0.0,0.333333
Zarza-Perrunal (La),42.0,1.0,3.0,1253.0,79.808460,239.425379,0.0,0.333333
